Fundamentals of tensorflow. 

#### OOPs in python

In [35]:
class SimpleClass():
    def __init__(self):
        print('Simple Class Constructor')
    
    def DoorBell(self):
        print("Tring")

In [36]:
x = SimpleClass()

Simple Class Constructor


In [37]:
x.DoorBell()

Tring


In [38]:
class SimpleClass_2():
    def __init__(self,name):
        print('Simple Class Constructor '+name)
    
    def clockSound(self):
        print("Tick Tock")

In [39]:
class ExtendedClass(SimpleClass_2):
    def __init__(self):
        super().__init__('Bhabani')
        print("Extended Class Constructor")

In [40]:
e = ExtendedClass()

Simple Class Constructor Bhabani
Extended Class Constructor


In [41]:
e.clockSound()

Tick Tock


In [1]:
class Operation():
    def __init__(self, input_nodes=[]):
        self.input_nodes = input_nodes
        self.output_nodes = []
        
        for nodes in input_nodes:
            nodes.output_nodes.append(self)
        
        _default_graph.operations.append(self)
    
    def compute(self):
        pass

In [2]:
class add(Operation):
    def __init__(self, x, y):
        super().__init__([x,y])
        
    def compute(self,x_var,y_var):
        self.inputs = [x_var, y_var]
        return x_var + y_var
        

In [3]:
class multiply(Operation):
    def __init__(self, a, b):
        super().__init__([a,b])
        
    def compute(self, a_var, b_var):
        self.inputs = [a_var, b_var]
        return a_var * b_var

In [4]:
class matmul(Operation):
    def __init__(self, a, b):
        super().__init__([a,b])
        
    def compute(self, a_mat, b_mat):
        self.inputs = [a_mat,b_mat]
        return a_mat.dot(b_mat)

In [5]:
class Placeholder():
    def __init__(self):
        self.output_nodes = []
        
        _default_graph.placeholders.append(self)

In [6]:
class Variable():
    def __init__(self, initial_value=None):
        self.value = initial_value
        self.output_nodes = []
        
        _default_graph.variables.append(self)

In [7]:
class Graph():
    def __init__(self):
        self.operations = []
        self.placeholders = []
        self.variables = []
    
    def set_as_default(self):
        global _default_graph
        _default_graph = self

##### Ax + b where A = 10, b = 1, Using x as 10

In [8]:
g = Graph()

In [9]:
g.set_as_default()

In [10]:
A = Variable(10)

In [11]:
x = Placeholder() # user input value

In [12]:
b = Variable(1)

In [13]:
y = multiply(A,x)

In [14]:
z = add(y,b)

## Session

In [33]:
import numpy as np

In [16]:
def traverse_postorder(operation):
    nodes_postorder = []
    def _recurse(node):
        if isinstance(node, Operation):
            for input_node in node.input_nodes:
                _recurse(input_node)
        nodes_postorder.append(node)
        
    _recurse(operation)
    return nodes_postorder

In [32]:
class Session():
    def run(self, operation, feed_dict={}):
        nodes_postorder = traverse_postorder(operation)
        
        for node in nodes_postorder:
            if type(node) == Placeholder:
                node.output = feed_dict[node]
                
            elif type(node) == Variable:
                node.output = node.value
                
            else: # Operation
                node.inputs = [input_node.output for input_node in node.input_nodes]
                node.output = node.compute(*node.inputs)
                
            #Convert lists into numpy arrays
            if type(node.output) == list:
                node.output = np.array(node.output)
                
        # Return the requested node value
        return operation.output
   

In [28]:
sess = Session()

In [19]:
result = sess.run(operation=z, feed_dict={x:10})

In [20]:
result

101

In [22]:
# Ax + b where A = 10, b = 1
10*10 + 1

101

In [34]:
# Matrix multiplication
C = Variable([[10,20], [30,40]])
x = Placeholder()
d = Variable([1,1])

In [36]:
y = matmul(C,x)

In [37]:
z = add(y,d)

In [39]:
matrix_result = sess.run(operation=z, feed_dict={x:10})

In [40]:
matrix_result

array([[101, 201],
       [301, 401]])